In [8]:
import sqlite3
import pandas as pd
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
import os

import plotly.graph_objs as go
import sys
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

To show the general palyer's trend over the year. Should I use the age to separate each play into one group, and then compute the average physical metric and compare them. So for player born in 1996, the average physical attributes will be. Same can be done for technical attributes.  
  
Also, I can introduce the nationality of the player. Or get the league of the player is playing. To see how the requirement for players in different league.  
  

In [9]:
# helper function to set up connection to database.
def _conn_db(path):
    database = os.path.join(path, 'database.sqlite')
    conn = sqlite3.connect(database)
    return conn
# helper function to get the data.
def _get_table(query,conn):
    df = pd.read_sql(query,conn)
    return df

In [10]:
path="."
conn = _conn_db(path)

In [19]:
# collect the data.
query = "select player_api_id,player_fifa_api_id,player_name,strftime('%Y',birthday) as birth_year from player;"
player_df = _get_table(query,conn)

In [14]:
query = "select * from League;"
league_df = _get_table(query, conn)

In [20]:
player_df.head()

,player_api_id,player_fifa_api_id,player_name,birth_year
0,505942,218353,Aaron Appindangoye,1992
1,155782,189615,Aaron Cresswell,1989
2,162549,186170,Aaron Doran,1991
3,30572,140161,Aaron Galindo,1982
4,23780,17725,Aaron Hughes,1979


In [15]:
league_df.head()

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A


In [21]:
query  = "select * from Team;"
team_df = _get_table(query, conn)

In [22]:
team_df.head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


In [23]:
query = "select * from Team_Attributes;"
team_att_df = _get_table(query, conn)
team_att_df.head()

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.0,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10 00:00:00,47,Balanced,41.0,Normal,54,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,4,77,8485,2010-02-22 00:00:00,70,Fast,NaN,Little,70,Long,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,5,77,8485,2011-02-22 00:00:00,47,Balanced,NaN,Little,52,Mixed,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover


In [24]:
query = "select * from Country;"
cont_df = _get_table(query, conn)

In [26]:
query = "select * from Match limit 100;"
match_df = _get_table(query, conn)

In [27]:
match_df.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [30]:
team_df.head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


TODO: 
1. Associate match, team , player,with country.

In [49]:
query = "select r3.name, r.team_long_name as home_team_name1, \
        r.team_short_name as home_team_name2,r2.team_long_name as away_team_name1, r2.team_short_name as \
        away_team_name2 from Match as l left join Team as r \
        on l.home_team_api_id = r.team_api_id \
        left join Team as r2 \
        on l.away_team_api_id=r2.team_api_id\
        left join League as r3\
        on l.league_id = r3.id;"
add_name_match_df = _get_table(query, conn)

In [51]:
add_name_match_df.head()

,name,home_team_name1,home_team_name2,away_team_name1,away_team_name2
0,Belgium Jupiler League,KRC Genk,GEN,Beerschot AC,BAC
1,Belgium Jupiler League,SV Zulte-Waregem,ZUL,Sporting Lokeren,LOK
2,Belgium Jupiler League,KSV Cercle Brugge,CEB,RSC Anderlecht,AND
3,Belgium Jupiler League,KAA Gent,GEN,RAEC Mons,MON
4,Belgium Jupiler League,FCV Dender EH,DEN,Standard de Liège,STL


To put country name back to Team. We only have country information in the $Match$ table, so we need to put back the name from match to team.

In [86]:
query = "select distinct l.team_long_name,l.team_fifa_api_id, l.team_short_name,r.country_id, \
        r2.name from Team as l left join Match as r on l.team_api_id = r.home_team_api_id\
        left join League as r2 on r2.country_id = r.country_id;"
new_team_df = _get_table(query, conn)

In [88]:
new_team_df.head()

,team_long_name,team_fifa_api_id,team_short_name,country_id,name
0,KRC Genk,673.0,GEN,1,Belgium Jupiler League
1,Beerschot AC,675.0,BAC,1,Belgium Jupiler League
2,SV Zulte-Waregem,15005.0,ZUL,1,Belgium Jupiler League
3,Sporting Lokeren,2007.0,LOK,1,Belgium Jupiler League
4,KSV Cercle Brugge,1750.0,CEB,1,Belgium Jupiler League


### To combine player's attributes with player general information.   
We have the player's birth year, player's physical data, and we can use the different year to show the general change of physical attributes. 

In [267]:
tech_att_name = ["ball_control","dribbling","short_passing","vision","crossing","long_passing","curve"]
physical_att_name = ['acceleration',"sprint_speed","agility","reactions","jumping","stamina","strength","balance"]
finishing_att_name = ["volleys","heading_accuracy","finishing","free_kick_accuracy","long_shots","shot_power","penalties"]
def_att_name = ["aggression","interceptions","positioning","marking","standing_tackle","sliding_tackle"]
gk_att_name = ['gk_diving', 'gk_handling', 'gk_kicking','gk_positioning', 'gk_reflexes']
general_att_name =["overall_rating","potential"]

FOCUS_DICT = {"technical":tech_att_name, "physical":physical_att_name, "finishing":finishing_att_name, "defensive":def_att_name,
            "goal_keeping":gk_att_name,"general":general_att_name}

COLORS = ["#BFEFFF", "#E3CF57", "#FF8C00", "#CD6600","#D1EEEE", "#D3D3D3", "#6495ED", "#66CD00",
                  "#006400", "#66CDAA", "#1C86EE"]

In [278]:
def _make_history_player_att(focus):
    """
    This function is to plot player's attributes overall change.
    param:
        focus: the category we want to see, it can be technical,physical, finishing, defensive,goal_keeping or general.
    return: 
        fig object to plot.
    """
    # get attributes name
    att_ls = FOCUS_DICT[focus]
    # get data, we need deduplicated aggregated player's data.
    df = _get_clean_paly_att_data(att_ls)
    # build traces.
    agg_df = df.groupby(by="birth_year").mean()[att_ls]
    data = []
    for ind,i in enumerate(agg_df):
        trace = go.Bar(
        x=agg_df.index.values,
        y=agg_df[i].values,
        name=i,
        marker=dict( color= COLORS[ind])
        )
        data.append(trace)
    # add layout
    layout = go.Layout(
    title="Player's {} abilities change over time.".format(focus),
    xaxis=dict(
        title='Birth Year',
        titlefont=dict(
            family='Old Standard TT, serif',
            size=18,
            color='Brown'
        ),
        showticklabels=True,
        tickangle=30,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        )
    ),
    yaxis=dict(
        range=[50, 75],
        title='Score Value',
        titlefont=dict(
            family='Old Standard TT, serif',
            size=18,
            color='black'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    legend=dict(
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

    fig = go.Figure(data=data, layout=layout)
    # return fig object.
    return fig

In [279]:
def _get_clean_paly_att_data(att_ls):
    """
    Get the joined table of player's attributes and play information. And aggregated on birth year.
    param:
        att_ls: the attribute names list.
    """
    command = [ "round(avg("+i+")) as "+i for i in att_ls]
    de_dup_paly_att_query = "select player_api_id, {} from Player_Attributes \
                            group by player_api_id".format(", ".join(command))
    query = "select r.player_name, r.player_api_id, strftime('%Y',r.birthday) as birth_year ,\
        l.{} from ({}) as l left join Player as \
        r on r.player_api_id = l.player_api_id;".format(", l.".join(att_ls), de_dup_paly_att_query) 
    new_player_att_df = _get_table(query, conn)
    return new_player_att_df

In [286]:
iplot(_make_history_player_att("technical"))

In [188]:
physical_att_name = ['acceleration',"sprint_speed","agility","reactions","jumping","stamina","strength","balance"]

# get the command for avg(att_name)

phy_command = [ "round(avg("+i+")) as "+i for i in physical_att_name]

# get deduplicated player_attributes.

de_dup_paly_att_query = "select player_api_id, {} from Player_Attributes group by player_api_id".format(", ".join(phy_command))

In [183]:
query

"select distinct r.player_name, r.player_api_id, strftime('%Y',r.birthday) as birth_year ,        l.acceleration, l.sprint_speed, l.agility, l.reactions, l.jumping, l.stamina, l.strength, l.balance from (select player_api_id, round(avg(acceleration)) as acceleration, round(avg(sprint_speed)) as sprint_speed, round(avg(agility)) as agility, round(avg(reactions)) as reactions, round(avg(jumping)) as jumping, round(avg(stamina)) as stamina, round(avg(strength)) as strength, round(avg(balance)) as balance from Player_Attributes group by player_api_id;) as l left join Player as         r on r.player_api_id = l.player_api_id"

In [189]:
query = "select r.player_name, r.player_api_id, strftime('%Y',r.birthday) as birth_year ,\
        l.{} from ({}) as l left join Player as \
        r on r.player_api_id = l.player_api_id;".format(", l.".join(physical_att_name),de_dup_paly_att_query) 
new_player_att_df = _get_table(query, conn)

In [198]:
phy_att_over_years = new_player_att_df.groupby(by="birth_year").mean()[physical_att_name]

In [287]:
phy_att_over_years

,acceleration,sprint_speed,agility,reactions,jumping,stamina,strength,balance
birth_year,,,,,,,,
1967,56.000000,69.000000,NaN,72.000000,NaN,76.000000,74.000000,NaN
1968,57.000000,62.000000,NaN,69.000000,NaN,61.000000,75.000000,NaN
1969,47.166667,50.000000,43.500000,63.333333,65.500000,53.500000,65.000000,53.500000
1970,49.583333,50.916667,53.400000,68.916667,70.200000,64.166667,72.083333,52.600000
1971,50.500000,50.583333,53.555556,65.500000,61.888889,61.333333,72.583333,58.555556
1972,53.090909,53.181818,45.400000,66.363636,65.500000,57.863636,71.863636,56.700000
1973,59.418605,59.837209,56.680000,67.511628,66.880000,62.325581,69.023256,60.640000
1974,59.051948,60.337662,55.750000,66.324675,63.770833,64.090909,69.597403,62.020833
1975,61.761062,61.743363,57.360000,67.796460,65.533333,64.619469,70.407080,64.760000


In [288]:
colors = ["#BFEFFF", "#E3CF57", "#FF8C00", "#CD6600","#D1EEEE", "#D3D3D3", "#6495ED", "#66CD00",
                  "#006400", "#66CDAA", "#1C86EE"]
data = []
for ind,i in enumerate(phy_att_over_years):
    trace = go.Bar(
    x=phy_att_over_years.index.values,
    y=phy_att_over_years[i].values,
    name=i,
    marker=dict(color= colors[ind])
    )
    data.append(trace)

In [289]:
layout = go.Layout(
    title="Player's Physical Attributes change over time.",
    xaxis=dict(
        title='Birth Year',
        titlefont=dict(
            family='Old Standard TT, serif',
            size=18,
            color='Brown'
        ),
        showticklabels=True,
        tickangle=30,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        )
    ),
    yaxis=dict(
        range=[55, 75],
        title='Score Value',
        titlefont=dict(
            family='Old Standard TT, serif',
            size=18,
            color='black'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    legend=dict(
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)

In [290]:
iplot(fig)

2. Get the line up data.

In [102]:
# select non-empty palyer information.
query = "select * from Match where home_player_1 is not null; "
match_with_player_info = _get_table(query, conn)

### Export as html file.  
To make the plot of player's attributes on technical, physical, defensive, finishing, goal keeping onto the html file.

In [283]:
html_doc = '''<!DOCTYPE html>
<html>
<head>
  <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
</head>

<div>
     <div style="display:flex">
     <h2><font color="black">EU Soccer Player's Attributes</font></h2>
     </div>
</div>
'''  

div_ls = []
for i in FOCUS_DICT.keys():
    fig = _make_history_player_att(i)
    _div = plot(fig, include_plotlyjs=False, output_type='div')
    html_doc += '''<h5><a id="1"></a>%(_div_title)s</h5> 
                      %(div)s
                ''' % {'div':_div, '_div_title':i}
    
html_doc += '''<hr>'''
html_doc +='''</body>
                  </html>'''

In [284]:
with open('EU_soccer.html', 'w') as f:
    f.write(html_doc)

In [285]:
!open EU_soccer.html